# How to add cross-thread persistence to your workflow (functional API)

!!! info "Prerequisites"

    This guide assumes familiarity with the following:
    
    - [Functional API](../../concepts/functional_api/)
    - [Persistence](../../concepts/persistence/)
    - [Memory](../../concepts/memory/)
    - [Chat Models](https://python.langchain.com/docs/concepts/chat_models/)

LangGraph allows you to persist data across **multiple threads**. For instance, you can store information about users (their names or preferences) in a shared memory and reuse them in the new conversational threads.

When creating a LangGraph workflow, you can set it up to store and retrieve memories by using the [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) interface:

```python
from langgraph.func import entrypoint, task
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()

@task
def my_task(task_input, *, store: BaseStore):
    # access the store from inside the task
    store.get(...)
    ...

@entrypoint(store=store)
def workflow(inputs):
    my_task(inputs).result()
    ...
```

In this guide, we will show how to construct and use a workflow that has a shared memory implemented using the [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) interface.

!!! note Note

    Support for the [`Store`](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) API that is used in this guide was added in LangGraph `v0.2.32`.

    Support for __index__ and __query__ arguments of the [`Store`](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) API that is used in this guide was added in LangGraph `v0.2.54`.

!!! tip "Note"

    If you need to add cross-thread persistence to a `StateGraph`, check out this [how-to guide](../cross-thread-persistence).

## Setup

First, let's install the required packages and set our API keys

In [2]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")
_set_env("OPENAI_API_KEY")

!!! tip "Set up [LangSmith](https://smith.langchain.com) for LangGraph development"

    Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started [here](https://docs.smith.langchain.com)

## Define store

In this example we will create a workflow that will be able to retrieve information about a user's preferences. We will do so by defining an `InMemoryStore` - an object that can store data in memory and query that data. We will then pass the store object to the [entrypoint()][langgraph.func.entrypoint] decorator. This allows each task in the workflow to access the store: when you define task functions, you can define `store` keyword argument, and LangGraph will automatically pass the store object you provided to the `entrypoint`.

When storing objects using the `Store` interface you define two things:

* the namespace for the object, a tuple (similar to directories)
* the object key (similar to filenames)

In our example, we'll be using `("memories", <user_id>)` as namespace and random UUID as key for each new memory.

Importantly, to determine the user, we will be passing `user_id` via the config keyword argument of the node function.

Let's first define our store!

In [4]:
from langgraph.store.memory import InMemoryStore
from langchain_openai import OpenAIEmbeddings

in_memory_store = InMemoryStore(
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "dims": 1536,
    }
)

## Create workflow

In [5]:
import uuid

from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import BaseMessage
from langgraph.func import entrypoint, task
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore


model = ChatAnthropic(model="claude-3-5-sonnet-latest")


# NOTE: we're passing the Store param to the task --
# this is the Store we pass to the entrypoint
@task
def call_model(
    messages: list[BaseMessage], config: RunnableConfig, *, store: BaseStore
):
    user_id = config["configurable"]["user_id"]
    namespace = ("memories", user_id)

    last_message = messages[-1]
    memories = store.search(namespace, query=str(last_message.content))
    info = "\n".join([d.value["data"] for d in memories])
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

    # Store new memories if the user asks the model to remember
    if "remember" in last_message.content.lower():
        memory = "User name is Bob"
        store.put(namespace, str(uuid.uuid4()), {"data": memory})

    response = model.invoke([{"type": "system", "content": system_msg}] + messages)
    return response


# NOTE: we're passing the store object here when creating a worfklow via entrypoint()
@entrypoint(checkpointer=MemorySaver(), store=in_memory_store)
def workflow(inputs: list[BaseMessage], *, previous: list[BaseMessage]):
    previous = previous or []
    inputs = add_messages(previous, inputs)
    response = call_model(inputs).result()
    return add_messages(inputs, response)

!!! note Note

    If you're using LangGraph Cloud or LangGraph Studio, you __don't need__ to pass store to the entrypoint decorator, since it's done automatically.

## Run the workflow!

Now let's specify a user ID in the config and tell the model our name:

In [6]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
input_message = {"type": "user", "content": "Hi! Remember: my name is Bob"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk[-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help you today?


In [7]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}
input_message = {"type": "user", "content": "what is my name?"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk[-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob.


We can now inspect our in-memory store and verify that we have in fact saved the memories for the user:

In [8]:
for memory in in_memory_store.search(("memories", "1")):
    print(memory.value)

{'data': 'User name is Bob'}


Let's now run the workflow for another user to verify that the memories about the first user are self contained:

In [9]:
config = {"configurable": {"thread_id": "3", "user_id": "2"}}
input_message = {"type": "user", "content": "what is my name?"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk[-1].pretty_print()

================================== Ai Message ==================================

I don't have any information about your name. I am starting fresh in this conversation and wasn't provided with any details about you. Feel free to introduce yourself if you'd like!
